# Variables and data types in python 

## Libraries and settings

In [1]:
# Libraries
import os
import random
import numpy as np
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

/Users/sivanujanselvarajah/Documents/zhaw/4.Semester/Scientific programming/scientific_programming/Week_02/exercises


## Creating an Excel file containing simulated data
(yes, this is not neccessary because we have Python, but it shows how we can work with Excel spread sheets from within Python)

In [2]:
# Empty list to store the data
col_01 = []
col_02 = []
col_03 = []
col_04 = []

# List with chicken breeds
breed = ['ISA Brown',
         'Plymouth Rock',
         'Barnevelder',
         'Australorp',
         'New Hampshire Red']

# Fill the empty lists with data
random.seed(10)
for i in range(1, 101):
    col_01.append(i)
    col_02.append(float(np.random.normal(2500, 250, 1)))
    col_03.append(breed[random.randint(0, 4)])
    col_04.append(random.randint(150, 365))

# Create a data frame from the lists
df = pd.DataFrame.from_dict({'chicken_id': col_01,
                             'weight': col_02,
                             'breed': col_03,
                             'eggs_per_year': col_04})

################################################################################

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter('chicken_data.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object
df.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False, index=False)

# Get the xlsxwriter objects from the dataframe writer object
workbook = writer.book
worksheet = writer.sheets['Sheet1']

# Get the dimensions of the dataframe
(max_row, max_col) = df.shape

# Create a list of column headers, to use in add_table()
column_settings = [{'header': column} for column in df.columns]

# Add the Excel table structure. Pandas will add the data
worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

# Make the columns wider for clarity
worksheet.set_column(0, max_col - 1, 20)

################################################################################

# Create pivot table
table = pd.pivot_table(df[['breed', 'eggs_per_year']],
                       index=['breed'],
                       values=['eggs_per_year'],
                       aggfunc=np.mean)

table.to_excel(writer, 
               sheet_name='Sheet2', 
               startrow=0, 
               header=True, 
               index=True)

# Include barchart
chart = workbook.add_chart({'type': 'bar', 'subtype': 'stacked'})

# Configure the first series.
chart.add_series({
    'name':       '=Sheet2!$B$1',
    'categories': '=Sheet2!$A$2:$A$6',
    'values':     '=Sheet2!$B$2:$B$6',
})

# Add a chart title and some axis labels.
chart.set_title ({'name': 'Average eggs per breed'})
chart.set_x_axis({'name': ''})
chart.set_y_axis({'name': ''})

# Set an Excel chart style.
chart.set_style(13)

# Insert the chart into the worksheet (with an offset).
worksheet.insert_chart('F1', chart, {'x_offset': 25, 'y_offset': 10})

# Insert .png - file
worksheet.insert_image('H17', '00_img_ipynb/chickens.jpg')

# Close the Pandas Excel writer and output the Excel file.
writer.close()

## Read data from Excel-File

In [4]:
# Read xlsx file
df = pd.read_excel('chicken_data.xlsx', sheet_name='Sheet1')
df

,chicken_id,weight,breed,eggs_per_year
0,1,2261.352256,New Hampshire Red,158
1,2,2866.760634,Australorp,273
2,3,2814.683914,New Hampshire Red,153
3,4,2484.408045,Plymouth Rock,268
4,5,2442.026205,Australorp,360
...,...,...,...,...
95,96,2325.479600,Plymouth Rock,314
96,97,2593.759307,New Hampshire Red,297
97,98,2485.364543,Australorp,156
98,99,2229.031712,ISA Brown,225


## Explore data types

In [5]:
# Explore data types (note that 'object' means categorical in pandas)
df.dtypes

chicken_id         int64
weight           float64
breed             object
eggs_per_year      int64
dtype: object

## Change data type of variable 'weight'

In [6]:
# Change data type (note that with .astype('int32'), the values are simply cutted!)
df['weight_new'] =  df['weight'].astype('int32')
print(df.dtypes, '\n')

df

chicken_id         int64
weight           float64
breed             object
eggs_per_year      int64
weight_new         int32
dtype: object 



,chicken_id,weight,breed,eggs_per_year,weight_new
0,1,2261.352256,New Hampshire Red,158,2261
1,2,2866.760634,Australorp,273,2866
2,3,2814.683914,New Hampshire Red,153,2814
3,4,2484.408045,Plymouth Rock,268,2484
4,5,2442.026205,Australorp,360,2442
...,...,...,...,...,...
95,96,2325.479600,Plymouth Rock,314,2325
96,97,2593.759307,New Hampshire Red,297,2593
97,98,2485.364543,Australorp,156,2485
98,99,2229.031712,ISA Brown,225,2229


## Create new variable 'breed_str' which has string as the data type

In [14]:
df['breed_str'] = pd.Series(['breed'], dtype="string")
df.dtypes

chicken_id                int64
weight                  float64
breed                    object
eggs_per_year             int64
weight_new                int32
breed_str        string[python]
eggs_cat               category
weight_kg               float64
dtype: object

## Create new categorical variable from 'eggs_per_year'

In [15]:
# Define labels for categories
labels = ['0 - 99', '100 - 199', '20 - 365']

# Create categories
df["eggs_cat"] = pd.cut(df['eggs_per_year'], bins=[0, 100, 200, 365], labels=labels)
df[['eggs_per_year', 'eggs_cat']].head(10)

,eggs_per_year,eggs_cat
0,158,100 - 199
1,273,20 - 365
2,153,100 - 199
3,268,20 - 365
4,360,20 - 365
5,317,20 - 365
6,158,100 - 199
7,275,20 - 365
8,169,100 - 199
9,340,20 - 365


## Create new numerical variable from 'weight'

In [9]:
# Create new variable
df['weight_kg'] = round(df['weight'] / 1000, 4)

# Show values
df

,chicken_id,weight,breed,eggs_per_year,weight_new,breed_str,eggs_cat,weight_kg
0,1,2261.352256,New Hampshire Red,158,2261,breed,100 - 199,2.2614
1,2,2866.760634,Australorp,273,2866,<NA>,20 - 365,2.8668
2,3,2814.683914,New Hampshire Red,153,2814,<NA>,100 - 199,2.8147
3,4,2484.408045,Plymouth Rock,268,2484,<NA>,20 - 365,2.4844
4,5,2442.026205,Australorp,360,2442,<NA>,20 - 365,2.4420
...,...,...,...,...,...,...,...,...
95,96,2325.479600,Plymouth Rock,314,2325,<NA>,20 - 365,2.3255
96,97,2593.759307,New Hampshire Red,297,2593,<NA>,20 - 365,2.5938
97,98,2485.364543,Australorp,156,2485,<NA>,100 - 199,2.4854
98,99,2229.031712,ISA Brown,225,2229,<NA>,20 - 365,2.2290


## Transform categorical variable 'breed' to matrix with binary (0/1) values

In [10]:
# Use the get_dummies() method from pandas for conversion
df_02 = pd.get_dummies(df, drop_first=False, columns=['breed'])
df_02

,chicken_id,weight,eggs_per_year,weight_new,breed_str,eggs_cat,weight_kg,breed_Australorp,breed_Barnevelder,breed_ISA Brown,breed_New Hampshire Red,breed_Plymouth Rock
0,1,2261.352256,158,2261,breed,100 - 199,2.2614,False,False,False,True,False
1,2,2866.760634,273,2866,<NA>,20 - 365,2.8668,True,False,False,False,False
2,3,2814.683914,153,2814,<NA>,100 - 199,2.8147,False,False,False,True,False
3,4,2484.408045,268,2484,<NA>,20 - 365,2.4844,False,False,False,False,True
4,5,2442.026205,360,2442,<NA>,20 - 365,2.4420,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,2325.479600,314,2325,<NA>,20 - 365,2.3255,False,False,False,False,True
96,97,2593.759307,297,2593,<NA>,20 - 365,2.5938,False,False,False,True,False
97,98,2485.364543,156,2485,<NA>,100 - 199,2.4854,True,False,False,False,False
98,99,2229.031712,225,2229,<NA>,20 - 365,2.2290,False,False,True,False,False


## Show data types of all generated variables

In [11]:
# Show data types
df_02.dtypes

chicken_id                          int64
weight                            float64
eggs_per_year                       int64
weight_new                          int32
breed_str                  string[python]
eggs_cat                         category
weight_kg                         float64
breed_Australorp                     bool
breed_Barnevelder                    bool
breed_ISA Brown                      bool
breed_New Hampshire Red              bool
breed_Plymouth Rock                  bool
dtype: object

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [12]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 23.3.0
Datetime: 2024-03-30 16:06:02
Python Version: 3.10.13
-----------------------------------
